<a href="https://colab.research.google.com/github/opf-ute/scripts/blob/master/acopf5barras_1106.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se quiere modelar y resolver el problema de flujo óptimo para el sistema representado por el unifilar de la figura. Se utilizará el modelo AC-OPF (BIM)



In [2]:
from IPython.display import Image
from IPython.core.display import HTML 

Image(url= "https://drive.google.com/uc?id=1zKqS2zehsZ2IxdM8sdNr7CHbdVfmTxtW", width=750)




In [0]:
#!pip install cvxpy
#!pip install git+https://github.com/cvxgrp/cvxpy.git

Bus Injection Model (BIM):

$$\mathbf{I}=\mathbf{Y}\mathbf{V}$$


$$\begin{bmatrix}
I_1\\
I_2\\
\vdots\\
I_N\end{bmatrix} = \begin{bmatrix}\ddots&&\\&Y_{kk}&Y_{ki}\\&&\ddots\end{bmatrix}\begin{bmatrix}
V_1\\
V_2\\
\vdots\\
V_N\end{bmatrix}$$


Donde $\mathbf{Y}$ es la matriz de admitancias con entradas

$$\left\{\begin{array}{lll}
Y_{kk}=&y_k+\sum_{i\sim k}y_{ki}&\\
Y_{ki}=&-y_{ki}&\text{si } k\sim i\\
Y_{ki}=&0&\text{en otro caso}
\end{array}\right.$$


Sea $Y_k$ la matriz que solamente contiene la k-esima fila de $Y$. Puede escribirse como

$$Y_k = E_k Y$$

Donde $E_k$ es cero salvo en la entrada $kk$, donde es $1$.

Se cumple
 
 $$Y^*_k=\left(\frac{Y^*_k+Y_k}{2}\right)+j\left(\frac{Y^*_k-Y_k}{2j}\right)=\Phi_k+j\Psi_k$$
 
 La potencia aparente inyectada en cada barra resulta
 
 
 $$\boxed{s_k = V\Phi_k V^*+jV^*\Psi_k V}$$

<hr>
 
 
Matriz de incidencia:

$$
A=\begin{bmatrix}
    0  & 0 & 0 & -1 & 1 \\
   -1  & 0 & 0 & 0 & 1 \\
   1  & 0 & 0 & -1 & 0 \\
   0  & 0 & -1 &1 & 0 \\
   1  & -1 & 0 & 0 & 0 \\
   0  & -1 & 1 & 0 & 0 \\
\end{bmatrix}
$$

Reactancias de linea:

$$
X=\begin{bmatrix}
    x_{54}  & 0 & 0 & 0 & 0 & 0\\
   0  & x_{51} & 0 & 0 & 0 & 0\\
   0  & 0 & x_{14} &  0 & 0 & 0 \\
   0  & 0 & 0 & x_{43} & 0 & 0 \\
   0  & 0 & 0 & 0 & x_{12} & 0 \\
   0  & 0 & 0 & 0 & 0 & x_{32} \\
\end{bmatrix}
$$

con $x_{12}=0.0281$, $x_{23}=0.0108$, $x_{34}=0.0297$, $x_{45}=0.0297$, $x_{14}=0.0304$, $x_{15}=0.0064$ (en p.u.).





In [0]:
Image(url= "https://drive.google.com/uc?id=1uoDeKu9d-sZhFM3R-2gmdplqNirh0cFf", width=750)


Luego el flujo de carga óptimo para esta red puede obtenerse con el siguiente codigo en CVXPY


In [0]:
# Import packages.
import cvxpy as cp
import numpy as np




In [13]:

# Modelo de cinco barras a modificar
Sbase = 100 # MVA
M=6
N=5
# Matriz de incidencia
A=np.array([[0, 0, 0, -1, 1],
            [-1, 0, 0, 0, 1],
            [1, 0, 0, -1, 0],
            [0, 0, -1, 1, 0],
            [1, -1, 0, 0, 0],
            [0, -1, 1, 0, 0] ]) 

# Reactancias en pu
x54 = 0.0297;
x51 = 0.0064;
x14 = 0.0304;
x43 = 0.0297;
x12 = 0.0281;
x32 = 0.0108;

X=np.array([[x54,0,0,0,0,0],
               [0,x51,0,0,0,0],
               [0,0,x14,0,0,0],
               [0,0,0,x43,0,0],
               [0,0,0,0,x12,0],
               [0,0,0,0,0,x32]])

R=X/10
Z=R+1j*X


#X=1j*np.array([[x54,0,0,0,0,0],
 #              [0,x51,0,0,0,0],
  #             [0,0,x14,0,0,0],
   #            [0,0,0,x43,0,0],
    #           [0,0,0,0,x12,0],
     #          [0,0,0,0,0,x32]])


#H=np.linalg.inv(X)@A
H=np.linalg.inv(Z)@A

Y = A.transpose()@H
Y = np.array(Y)


# ----------------------------
# Caracteristicas del problema
# ----------------------------
d = np.array([0,300,300,400,0]) / Sbase # Demanda
d=d+0.75*d*1j
plmax = np.array([240,400,400,400,400,400]) / Sbase # Limite lineas

gmax = np.array([40,170,520,200,600]) / Sbase # Limite generacion
qmax = np.array([40,170,520,200,600]) / Sbase # Limite generacion
c = np.array([14,15,30,40,10]) # Costo

# G2pg pasa de generadores a generacion por barra:
# pg = G2pg * g
G2pg=np.array([[1,1,0,0,0],
               [0,0,0,0,0],
               [0,0,1,0,0],
               [0,0,0,1,0],
               [0,0,0,0,1]])



# --------------------------
# Crear matrices hermiticas
# --------------------------

# Asociadas al balance de cada nodo
# ---------------------------------
E = np.zeros([5,5,5])
y = np.zeros([5,5,5], dtype=np.complex64)
Phi = np.zeros([5,5,5], dtype=np.complex64)
Psi = np.zeros([5,5,5], dtype=np.complex64)
                                  

for k in range(0,5):
  E[k,k,k] = 1 # Todo ceros salvo en el kk-esimo elemento
  y[k] = E[k] @ Y # y[k] tiene en su k-esima fila la k-esima fila de Y
  Phi[k] = .5 * (np.matrix(y[k]) + np.matrix(y[k]).H) 
  Psi[k] = .5 /(1j) * (-np.matrix(y[k]) + np.matrix(y[k]).H)



# ---------------------------------------  
# Asociadas a las restricciones en lineas
# ---------------------------------------
D = [(4,3), (4,0), (0,3), (3,2), (0,1), (2,1)] #Linea 1 conecta (4->3); Linea 2 conecta (4->0), etc.

ek = [[0,0,0,0,1], # linea 1 sale de barra 5
      [0,0,0,0,1], # linea 2 sale de barra 5
      [1,0,0,0,0], # Linea 3 sale de barra 1 ...
      [0,0,0,1,0],
      [1,0,0,0,0],
      [0,0,1,0,0]]

# Matrices de restricciones en las lineas
M = np.zeros([6,5,5], dtype=np.complex64)                 
N = np.zeros([6,5,5], dtype=np.complex64)

for l in range(0,6):
  Yki_aster = np.conj(Y[D[l]]) * np.outer(A[l], ek[l])  
  M[l] = .5 * (np.matrix(Yki_aster) + np.matrix(Yki_aster).H)  
  N[l] = .5 * (np.matrix(Yki_aster) - np.matrix(Yki_aster).H) / 1j  



                 
#for k in range(0,5):
#  print('Phi[',k,'] es hermitica : ', np.allclose(np.matrix(Phi[k]),np.matrix(Phi[k]).H))

# C0 = c[0]*Phi[0] + c[1]*Phi[1] + c[2]*Phi[2] + c[3]*Phi[3] + c[4]*Phi[4]
                               

  
  
  
# ------------------
# Variables de CVXPY
# ------------------
pb = cp.Variable(5) # potencia neta inyectada en las barras = pg-d
qb = cp.Variable(5) # potencia neta inyectada en las barras = pg-d
g = cp.Variable(5) # potencia de cada generador 
q = cp.Variable(5) # potencia de cada generador 
pl = cp.Variable(6) # potencia por las líneas
pg = cp.Variable(5) # potencias inyectadas por los generadores en las barras; pg = G2pg*g
qg = cp.Variable(5) # potencias inyectadas por los generadores en las barras; qg = G2pg*q


W = cp.Variable((5,5), hermitian=True) # Matriz hermitica, (con suerte) Rank=1
#X = cp.Variable((5,5))
#Y = cp.Variable((5,5))

#cost = cp.trace(A*X - B*Y)
cost = (c*g)

constraints = [G2pg*g == pg, G2pg*q == qg, pb==pg-cp.real(d),qb==qg-cp.imag(d), g>=0 , g<=gmax, q<=qmax, q>=-qmax,  W>>0]#, pl <= plmax, pl >= -plmax]
#constraints.append(X == cp.real(W))
#constraints.append(Y == cp.imag(W))


# Balance por nodo
for k in range(0,5):
  Phir = np.real(Phi[k])
  Phii = np.imag(Phi[k])
  Psir = np.real(Psi[k])
  Psii = np.imag(Psi[k])
  #la siguiente línea implementa traza(Phi*W)=pb[k]
  constraints.append(cp.trace(Phir*cp.real(W) - Phii*cp.imag(W)) == pb[k])
  #constraints.append(cp.trace(Phir*X - Phii*Y) == pb[k])
  constraints.append(cp.trace(Psir*cp.real(W) - Psii*cp.imag(W)) == qb[k])
  

# Restricciones de linea
constraints.append(pl <= plmax)
constraints.append(-pl <= plmax)
#constraints.append(ql <= qlmax)
#constraints.append(-ql <= qlmax)
for l in range(0,6):
  Mr = np.real(M[l])
  Mi = np.imag(M[l])
  #Nr = np.real(N[l])
  #Ni = np.imag(N[l])
  constraints.append(cp.trace(Mr*cp.real(W) - Mi*cp.imag(W)) == -pl[l])
  #constraints.append(cp.trace(Mr*X - Mi*Y) == -pl[k])
  #constraints.append(cp.trace(Nr*cp.real(W) - Ni*cp.imag(W)) == -ql[l])
  #np.trace(N[k] @ W.value)
  
  
# for k in range(0,5):
#   constraints.append(cp.trace(E[k]*X)<=1.1)
#   constraints.append(cp.trace(E[k]*X)>=.9)  
  
# Restricciones de tension
#constraints.append(cp.diag(X)<=1.1**2)
#constraints.append(cp.diag(X)>=.9**2)
constraints.append(cp.diag(cp.real(W))<=1.1**2)
constraints.append(cp.diag(cp.real(W))>=.9**2)

prob = cp.Problem(cp.Minimize(cost),constraints)

prob.solve(verbose=True) #Poniendo verbose=False se oculta el Solver



# Inyeccion de reactiva
qb = np.zeros(5)
for k in range(0,5):
  qb[k] = np.trace(Psi[k]@W.value)

# Transporte de reactiva (lineas)
ql = np.zeros(5)
for k in range(0,5):
  ql[k] = np.trace(N[k] @ W.value)

# ------------------
# MOSTRAR RESULTADOS
# ------------------
np.set_printoptions(precision=0)
np.set_printoptions(suppress=True)
print()
print()
print('RESULTADOS:')
print("El costo óptimo es", prob.value*100)
print()
print('Generacion activa optima', g.value*100)
print('Generacion reactiva optima', q.value*100)

print('------')
print('Barras')
print('------')
print('Generacion activa', pg.value*100)
print('Generacion reactiva', qg.value*100)
print("Inyeccion P", pb.value*100)
print('Inyeccion Q', qb*100)

print('------')
print('Lineas')
print('------')
print('P:', pl.value*100)
print('Q:', ql*100)

  


#print('Multiplicadores de Lagrange:')
#for k in range(0,7):
#  print(constraints[k].dual_value)
#print(np.allclose(W.value,np.matrix(W.value).H))
#print(W.value)
print('')
print('-------------------')
print('Checkeo: W rango 1:')
print('-------------------')


print('Valores propios de W:')
vapW, eigW = np.linalg.eig(W.value)
print('Valores propios',vapW)


z=np.where(np.abs(vapW)>1e-2)

print('Unico valor propio no nulo:')
print(vapW[z])

voltaje = eigW[:,0]*np.sqrt(vapW[0])



voltaje_modulo = np.abs(voltaje)
tita = np.angle(voltaje)*180/np.pi # Pasaje a grados
print()
print('Barra\t V(pu)\t Angulo(grados)')
for i in range(0,5):
  print(i,'\t','%.3f'%voltaje_modulo[i],'\t','%.3f'%tita[i])


----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 279
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 76, constraints m = 133
Cones:	primal zero / dual free vars: 36
	linear vars: 42
	sd vars: 55, sd blks: 1
Setup time: 7.27e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.67e+20  4.59e+20  1.00e+00 -6.11e+22  8.78e+21  4.48e+22  1.03e-02 
   100| 1.75e-01  9.36e-02  5.72e-02  1.70e+02  1.51e+02  7.79e-14  1.91e-02 
   200| 2.33e-01  1.27e-01  7.64e-02  1.29e+02  1.50e+02  1.05e-13  2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:191: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:196: ComplexWarning: Casting complex values to real discards the imaginary part


a) Reflexionar sobre qué sucedería si se modificaran los costos de generación y luego correr el problema con nuevos costos.

b) Reflexionar sobre qué sucedería si se modificaran los límites de las líneas y luego correr el problema con nuevos costos.



In [0]:
#Escribir código aquí